# 🚀 Lab 1-3: Batch 데이터 파이프라인

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 90분 (1.5시간) |
| **난이도** | ⭐⭐⭐ |
| **목표** | AWS S3 기반 Data Lake 구축 및 Batch 데이터 처리 파이프라인 구현 |

## 🎯 학습 목표

이 실습을 통해 다음을 학습합니다:
- **AWS S3 기반 Data Lake 아키텍처** 이해 및 구축
- **ETL Pipeline** 설계 및 구현
- **데이터 품질 관리** 자동화
- **대규모 Batch 데이터 처리** (Pandas/AWS Wrangler 활용)
- **Bronze → Silver → Gold Layer** 데이터 흐름 이해

---

## 🏗️ 실습 구조

```
Lab 1-3: Batch Data Pipeline (90분)
├── Part 1: ETL Pipeline (45분)
│   ├── S3 Data Lake 구축
│   ├── 샘플 데이터 생성 (1000명 고객)
│   ├── ETL 파이프라인 실행
│   └── 데이터 품질 검증
└── Part 2: Batch Processing (45분)
    ├── Silver Layer 데이터 읽기
    ├── Batch 데이터 집계 (Pandas)
    ├── 결과 분석
    └── Gold Layer 저장
```

---

## ⚠️ Cell 0: AWS 자격 증명 설정 (필수!)

**실습 시작 전 반드시 AWS 자격 증명을 설정하세요!**

In [ ]:
# ============================================================
# AWS 자격 증명 설정 (필수!)
# ============================================================
import os

# ⚠️ 본인의 사용자 번호로 변경하세요!
USER_NUM = "01"  # 예: "01", "02", "03"...

# AWS 자격 증명 설정 (강사가 제공한 키를 입력하세요!)
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'          # 변경 필요!
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'      # 변경 필요!
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-2'
os.environ['USER_NUM'] = USER_NUM

print(f"✅ 환경 변수 설정 완료!")
print(f"   사용자 번호: {USER_NUM}")
print(f"   AWS 리전: ap-northeast-2 (서울)")

In [ ]:
# ============================================================
# AWS S3 연결 테스트
# ============================================================
import boto3

s3 = boto3.client('s3')

try:
    response = s3.list_buckets()
    print(f"✅ AWS S3 연결 성공! 버킷 개수: {len(response['Buckets'])}")
    print("\n📦 기존 버킷 목록:")
    for bucket in response['Buckets'][:5]:  # 최대 5개만 표시
        print(f"   - {bucket['Name']}")
    if len(response['Buckets']) > 5:
        print(f"   ... 외 {len(response['Buckets']) - 5}개")
except Exception as e:
    print(f"❌ AWS S3 연결 실패: {e}")
    print("\n🔧 해결 방법:")
    print("   1. AWS 자격 증명을 올바르게 입력했는지 확인하세요.")
    print("   2. 강사에게 액세스 키를 요청하세요.")

---

# 🎯 Part 1: ETL Pipeline (45분)

## 학습 목표
- S3 Data Lake 구조 이해 (Bronze/Silver/Gold 레이어)
- Extract-Transform-Load 프로세스 구현
- 데이터 품질 검증 자동화

## Step 1-1: 필수 패키지 임포트 및 환경 설정

In [ ]:
# ============================================================
# 필수 패키지 임포트
# ============================================================
import os
import pandas as pd
import numpy as np
import boto3
from datetime import datetime, timedelta
import awswrangler as wr

print("✅ 패키지 임포트 완료!")
print(f"   pandas: {pd.__version__}")
print(f"   numpy: {np.__version__}")
print(f"   awswrangler: {wr.__version__}")

In [ ]:
# ============================================================
# 환경 변수 및 S3 경로 설정
# ============================================================
print("=" * 60)
print("ETL 파이프라인 - 환경 설정")
print("=" * 60)

# 환경 변수 읽기
USER_NUM = os.getenv('USER_NUM', '01')
AWS_REGION = os.getenv('AWS_DEFAULT_REGION', 'ap-northeast-2')

# S3 버킷 이름 생성 (사용자별 고유)
BUCKET_NAME = f"mlops-training-user{USER_NUM}"

# Data Lake 레이어 정의
BRONZE_LAYER = f"s3://{BUCKET_NAME}/raw/"          # 원본 데이터
SILVER_LAYER = f"s3://{BUCKET_NAME}/processed/"   # 정제된 데이터
GOLD_LAYER = f"s3://{BUCKET_NAME}/curated/"       # 집계된 데이터

print(f"\n📌 설정 정보:")
print(f"   사용자: {USER_NUM}")
print(f"   버킷: {BUCKET_NAME}")
print(f"   리전: {AWS_REGION}")
print(f"\n📁 Data Lake 구조:")
print(f"   Bronze Layer (원본): {BRONZE_LAYER}")
print(f"   Silver Layer (정제): {SILVER_LAYER}")
print(f"   Gold Layer (집계):   {GOLD_LAYER}")

## Step 1-2: S3 Data Lake 구조 생성

In [ ]:
# ============================================================
# STEP 1: S3 Data Lake 구조 생성
# ============================================================
print("\n" + "=" * 60)
print("STEP 1: S3 Data Lake 생성")
print("=" * 60)

# boto3 S3 클라이언트 생성
s3_client = boto3.client('s3', region_name=AWS_REGION)

try:
    # S3 버킷 생성
    s3_client.create_bucket(
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': AWS_REGION}
    )
    print(f"✅ 버킷 생성 완료: {BUCKET_NAME}")
except s3_client.exceptions.BucketAlreadyOwnedByYou:
    print(f"✅ 버킷이 이미 존재함: {BUCKET_NAME}")
except Exception as e:
    print(f"⚠️  버킷 생성 오류: {e}")
    print("   (다른 사용자가 이미 사용 중인 버킷 이름일 수 있습니다)")

print("\n📁 Data Lake 구조:")
print(f"  Bronze Layer (원본): {BRONZE_LAYER}")
print(f"  Silver Layer (정제): {SILVER_LAYER}")
print(f"  Gold Layer (집계):   {GOLD_LAYER}")

## Step 1-3: 샘플 고객 데이터 생성 (1000명)

In [ ]:
# ============================================================
# STEP 2: 샘플 데이터 생성
# ============================================================
print("\n" + "=" * 60)
print("STEP 2: 샘플 데이터 생성")
print("=" * 60)

# 랜덤 시드 고정 (재현 가능성)
np.random.seed(42)

# 고객 데이터 생성
num_customers = 1000
customer_ids = list(range(1, num_customers + 1))
names = [f"Customer_{i}" for i in customer_ids]
ages = np.random.randint(18, 70, num_customers)
emails = [f"user{i}@example.com" for i in customer_ids]
cities = np.random.choice(['Seoul', 'Busan', 'Incheon', 'Daegu'], num_customers)

# 가입일 생성 (최근 1년 내)
join_dates = [
    datetime.now() - timedelta(days=np.random.randint(1, 365))
    for _ in range(num_customers)
]

print(f"✅ 기본 데이터 생성 완료: {num_customers}명")

In [ ]:
# ============================================================
# ⚠️ 의도적으로 데이터 품질 이슈 추가 (10%)
# ============================================================
print("\n⚠️ 데이터 품질 이슈 추가 (ETL 실습을 위함)...")

issue_indices = np.random.choice(num_customers, size=100, replace=False)

# 1. Null 값 추가 (33개)
for idx in issue_indices[:33]:
    emails[idx] = None

# 2. 중복 추가 (33개)
for idx in issue_indices[33:66]:
    customer_ids[idx] = customer_ids[0]  # 첫 번째 ID와 동일하게

# 3. 잘못된 이메일 형식 추가 (34개)
for idx in issue_indices[66:]:
    emails[idx] = f"invalid_{idx}"  # @ 없는 잘못된 형식

# DataFrame 생성
df_customers = pd.DataFrame({
    'customer_id': customer_ids,
    'name': names,
    'age': ages,
    'email': emails,
    'city': cities,
    'join_date': join_dates
})

print(f"\n✅ {len(df_customers)}명 고객 데이터 생성 완료")
print(f"\n📊 데이터 품질 이슈 (의도적으로 추가됨):")
print(f"   - Null 값: {df_customers['email'].isnull().sum()}개")
print(f"   - 중복 ID: {df_customers['customer_id'].duplicated().sum()}개")
print(f"   - 잘못된 형식: {len([e for e in emails if e and '@' not in str(e)])}개")

In [ ]:
# ============================================================
# 데이터 미리보기
# ============================================================
print("\n📋 생성된 데이터 미리보기:")
df_customers.head(10)

In [ ]:
# ============================================================
# 데이터 정보 확인
# ============================================================
print("📊 데이터 정보:")
print(df_customers.info())
print("\n📈 통계 요약:")
df_customers.describe()

## Step 1-4: Bronze Layer에 원본 데이터 저장

In [ ]:
# ============================================================
# Bronze Layer에 저장
# ============================================================
print("\n" + "=" * 60)
print("Bronze Layer에 원본 데이터 저장")
print("=" * 60)

bronze_path = BRONZE_LAYER + "customers_raw/"

try:
    wr.s3.to_parquet(
        df=df_customers,
        path=bronze_path,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ Bronze Layer 저장 완료: {bronze_path}")
except Exception as e:
    print(f"❌ 저장 실패: {e}")

## Step 1-5: ETL 프로세스 - Extract (추출)

In [ ]:
# ============================================================
# STEP 3: ETL 프로세스
# ============================================================
print("\n" + "=" * 60)
print("STEP 3: ETL 처리")
print("=" * 60)

# 3-1. Extract (추출): Bronze Layer에서 데이터 읽기
print("\n📥 Extract: Bronze Layer에서 데이터 읽기...")
df_raw = wr.s3.read_parquet(bronze_path)
print(f"✅ Bronze Layer에서 {len(df_raw)}행 로드 완료")

## Step 1-6: ETL 프로세스 - Transform (변환)

In [ ]:
# ============================================================
# 3-2. Transform (변환): 데이터 검증
# ============================================================
print("\n🔍 Transform: 데이터 검증 중...")

# 품질 이슈 카운트
null_emails = df_raw['email'].isnull().sum()
duplicate_ids = df_raw['customer_id'].duplicated().sum()
invalid_emails = len(df_raw[df_raw['email'].notna() & ~df_raw['email'].str.contains('@', na=False)])

print(f"\n⚠️ 품질 이슈 발견:")
print(f"   - Null 이메일: {null_emails}개")
print(f"   - 중복 ID: {duplicate_ids}개")
print(f"   - 잘못된 이메일 형식: {invalid_emails}개")
print(f"   - 총 이슈: {null_emails + duplicate_ids + invalid_emails}개")

In [ ]:
# ============================================================
# 3-2. Transform (변환): 데이터 정제
# ============================================================
print("\n🔧 Transform: 데이터 정제 중...")

# 데이터 정제
df_clean = df_raw.copy()

# 1) Null 이메일 제거
before_count = len(df_clean)
df_clean = df_clean[df_clean['email'].notna()]
print(f"   ✅ Null 이메일 제거: {before_count - len(df_clean)}행")

# 2) 중복 제거 (customer_id 기준)
before_count = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['customer_id'])
print(f"   ✅ 중복 제거: {before_count - len(df_clean)}행")

# 3) 잘못된 이메일 형식 제거
before_count = len(df_clean)
df_clean = df_clean[df_clean['email'].str.contains('@', na=False)]
print(f"   ✅ 잘못된 이메일 형식 제거: {before_count - len(df_clean)}행")

print(f"\n✅ 정제된 데이터: {len(df_clean)}행")

In [ ]:
# ============================================================
# 3-2. Transform (변환): 새로운 컬럼 추가
# ============================================================
print("\n📊 Transform: 데이터 변환 (새 컬럼 추가)...")

# 1) 이메일 도메인 추출
df_clean['email_domain'] = df_clean['email'].str.split('@').str[1]
print(f"   ✅ email_domain 컬럼 추가")

# 2) 나이대 그룹 생성
def age_to_group(age):
    """나이를 나이대 그룹으로 변환"""
    if age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    else:
        return '60+'

df_clean['age_group'] = df_clean['age'].apply(age_to_group)
print(f"   ✅ age_group 컬럼 추가")

# 변환된 데이터 미리보기
print("\n📋 변환된 데이터 미리보기:")
df_clean.head()

## Step 1-7: ETL 프로세스 - Load (적재)

In [ ]:
# ============================================================
# 3-3. Load (적재): Silver Layer에 저장
# ============================================================
print("\n" + "=" * 60)
print("Load: Silver Layer에 정제된 데이터 저장")
print("=" * 60)

silver_path = SILVER_LAYER + "customers_cleaned/"
print(f"\n💾 Silver Layer에 저장 중...")

try:
    wr.s3.to_parquet(
        df=df_clean,
        path=silver_path,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ 저장 완료: {silver_path}")
except Exception as e:
    print(f"❌ 저장 실패: {e}")

## Step 1-8: 데이터 품질 리포트

In [ ]:
# ============================================================
# 데이터 품질 리포트
# ============================================================
print("\n" + "=" * 60)
print("📊 데이터 품질 리포트")
print("=" * 60)

# 품질 점수 계산
total_rows = len(df_raw)
cleaned_rows = len(df_clean)
quality_score = (cleaned_rows / total_rows) * 100

print(f"\n🎯 데이터 품질 점수: {quality_score:.1f}%")
print(f"\n📈 처리 결과:")
print(f"   원본 데이터: {total_rows}행")
print(f"   정제된 데이터: {cleaned_rows}행")
print(f"   제거된 데이터: {total_rows - cleaned_rows}행")

# 품질 이슈 요약
print(f"\n✅ 품질 이슈 해결:")
print(f"   ✅ Null 값 처리 완료")
print(f"   ✅ 중복 제거 완료")
print(f"   ✅ 데이터 형식 검증 완료")

print("\n" + "=" * 60)
print("✅ Part 1: ETL 파이프라인 완료!")
print("=" * 60)

## Step 1-9: S3 결과 확인

In [ ]:
# ============================================================
# S3 결과 확인
# ============================================================
print("\n📁 S3 저장 결과 확인:")

# Bronze Layer 확인
bronze_files = wr.s3.list_objects(bronze_path)
print(f"\n🥉 Bronze Layer ({bronze_path}):")
for f in bronze_files[:3]:
    print(f"   - {f.split('/')[-1]}")

# Silver Layer 확인
silver_files = wr.s3.list_objects(silver_path)
print(f"\n🥈 Silver Layer ({silver_path}):")
for f in silver_files[:3]:
    print(f"   - {f.split('/')[-1]}")

---

# 🎯 Part 2: Batch Processing (45분)

## 학습 목표
- 대규모 데이터 Batch 처리
- Pandas 및 AWS Wrangler 활용
- Gold Layer 데이터 생성

## Step 2-1: Silver Layer 데이터 읽기

In [ ]:
# ============================================================
# Part 2: Batch Processing
# ============================================================
print("=" * 60)
print("🚀 BATCH 데이터 처리 (Pandas)")
print("=" * 60)

print(f"\n📌 설정 정보:")
print(f"   사용자: {USER_NUM}")
print(f"   버킷: {BUCKET_NAME}")
print(f"   리전: {AWS_REGION}")

In [ ]:
# ============================================================
# Silver Layer 데이터 읽기
# ============================================================
silver_path = f"s3://{BUCKET_NAME}/processed/customers_cleaned/"
print(f"\n{'=' * 60}")
print(f"📥 Silver Layer 데이터 읽기")
print(f"{'=' * 60}")
print(f"경로: {silver_path}")

try:
    # AWS Wrangler로 Parquet 읽기
    df = wr.s3.read_parquet(silver_path)
    print(f"\n✅ {len(df)}행 로드 완료")
    print(f"\n📊 스키마:")
    print(df.dtypes)
    
except Exception as e:
    print(f"❌ 데이터 읽기 실패: {e}")
    print(f"\n가능한 원인:")
    print(f"   1. Silver Layer 데이터가 없음 (Part 1을 먼저 실행하세요)")
    print(f"   2. S3 권한 문제")
    print(f"   3. 경로 오류")

In [ ]:
# ============================================================
# Silver Layer 데이터 미리보기
# ============================================================
print("\n📋 Silver Layer 데이터 미리보기:")
df.head(10)

## Step 2-2: 데이터 분석

In [ ]:
# ============================================================
# 데이터 분석
# ============================================================
print("\n" + "=" * 60)
print("📊 데이터 분석")
print("=" * 60)

In [ ]:
# ============================================================
# 1️⃣ 도시별 고객 수 집계
# ============================================================
print("\n1️⃣ 도시별 고객 수:")
city_counts = df.groupby('city').size().reset_index(name='count')
city_counts = city_counts.sort_values('count', ascending=False)
print(city_counts.to_string(index=False))

In [ ]:
# ============================================================
# 2️⃣ 나이대별 분포
# ============================================================
print("\n2️⃣ 나이대별 분포:")
age_group_counts = df.groupby('age_group').size().reset_index(name='count')
age_group_counts = age_group_counts.sort_values('age_group')
print(age_group_counts.to_string(index=False))

In [ ]:
# ============================================================
# 3️⃣ 이메일 도메인별 분포 (Top 5)
# ============================================================
print("\n3️⃣ 이메일 도메인 Top 5:")
domain_counts = df.groupby('email_domain').size().reset_index(name='count')
domain_counts = domain_counts.sort_values('count', ascending=False).head(5)
print(domain_counts.to_string(index=False))

In [ ]:
# ============================================================
# 4️⃣ 통계 요약
# ============================================================
print("\n4️⃣ 통계 요약:")
stats = pd.DataFrame({
    'avg_age': [df['age'].mean()],
    'max_age': [df['age'].max()],
    'min_age': [df['age'].min()],
    'total_customers': [len(df)]
})
print(stats.to_string(index=False))

## Step 2-3: 데이터 시각화 (선택사항)

In [ ]:
# ============================================================
# 데이터 시각화 (선택사항)
# ============================================================
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # 도시별 고객 수 차트
    axes[0].bar(city_counts['city'], city_counts['count'], color='steelblue')
    axes[0].set_title('Number of customers by city')
    axes[0].set_xlabel('City')
    axes[0].set_ylabel('Number of customers')
    
    # 나이대별 분포 차트
    axes[1].bar(age_group_counts['age_group'], age_group_counts['count'], color='coral')
    axes[1].set_title('Distribution by age group')
    axes[1].set_xlabel('Age group')
    axes[1].set_ylabel('Number of customers')
    
    plt.tight_layout()
    plt.show()
    print("✅ 시각화 완료!")
    
except ImportError:
    print("⚠️ matplotlib가 설치되지 않아 시각화를 건너뜁니다.")
    print("   설치: pip install matplotlib")

## Step 2-4: Gold Layer에 결과 저장

In [ ]:
# ============================================================
# Gold Layer에 결과 저장
# ============================================================
gold_path = f"s3://{BUCKET_NAME}/curated/analysis/"
print("\n" + "=" * 60)
print(f"💾 Gold Layer에 결과 저장")
print(f"{'=' * 60}")
print(f"경로: {gold_path}")

try:
    # 1) 도시별 분석 저장
    city_output = gold_path + "city_analysis/"
    wr.s3.to_parquet(
        df=city_counts,
        path=city_output,
        dataset=True,
        mode='overwrite'
    )
    print(f"\n✅ 도시별 분석 저장: {city_output}")
    
    # 2) 나이대별 분석 저장
    age_output = gold_path + "age_analysis/"
    wr.s3.to_parquet(
        df=age_group_counts,
        path=age_output,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ 나이대별 분석 저장: {age_output}")
    
    # 3) 도메인별 분석 저장
    domain_output = gold_path + "domain_analysis/"
    wr.s3.to_parquet(
        df=domain_counts,
        path=domain_output,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ 도메인별 분석 저장: {domain_output}")
    
    # 4) 통계 요약 저장
    stats_output = gold_path + "statistics/"
    wr.s3.to_parquet(
        df=stats,
        path=stats_output,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ 통계 요약 저장: {stats_output}")
    
    # 5) 메타데이터 저장
    metadata = pd.DataFrame({
        'processed_at': [datetime.now()],
        'total_rows': [len(df)],
        'source': [silver_path]
    })
    metadata_output = gold_path + "metadata/"
    wr.s3.to_parquet(
        df=metadata,
        path=metadata_output,
        dataset=True,
        mode='overwrite'
    )
    print(f"✅ 메타데이터 저장: {metadata_output}")
    
except Exception as e:
    print(f"❌ 저장 실패: {e}")

## Step 2-5: 최종 결과 확인

In [ ]:
# ============================================================
# 최종 결과 확인
# ============================================================
print("\n" + "=" * 60)
print("✅ BATCH 데이터 처리 완료!")
print("=" * 60)

print(f"\n📁 결과 위치: {gold_path}")
print(f"\n🔍 S3에서 확인:")
print(f"   aws s3 ls s3://{BUCKET_NAME}/curated/analysis/ --recursive")

print(f"\n📊 처리된 데이터:")
print(f"   - 도시별 분석: {len(city_counts)}개 도시")
print(f"   - 나이대별 분석: {len(age_group_counts)}개 그룹")
print(f"   - 도메인별 분석: Top {len(domain_counts)}")
print(f"   - 통계: 평균 나이 {stats['avg_age'].values[0]:.1f}세")

In [ ]:
# ============================================================
# Gold Layer 결과 검증
# ============================================================
print("\n📥 Gold Layer 결과 검증:")

# 도시별 분석 읽기
city_df = wr.s3.read_parquet(gold_path + "city_analysis/")
print(f"\n🏙️ 도시별 고객 수:")
print(city_df)

# 나이대별 분석 읽기
age_df = wr.s3.read_parquet(gold_path + "age_analysis/")
print(f"\n👤 나이대별 분포:")
print(age_df)

---

# ✅ 실습 완료 체크리스트

## Part 1: ETL Pipeline (45분)
- [ ] S3 버킷 생성 완료
- [ ] 샘플 데이터 생성 완료 (1000명)
- [ ] ETL 파이프라인 실행 성공
- [ ] 데이터 품질 점수 ≥ 90%
- [ ] Silver Layer Parquet 저장 완료

## Part 2: Batch Processing (45분)
- [ ] Batch processing 스크립트 이해
- [ ] Silver Layer 데이터 로드 성공
- [ ] 도시별 분석 결과 확인
- [ ] 나이대별 분석 결과 확인
- [ ] Gold Layer 저장 완료
- [ ] S3에서 결과 파일 확인

In [ ]:
# ============================================================
# 전체 Data Lake 구조 확인
# ============================================================
print("\n" + "=" * 60)
print("🏆 전체 Data Lake 구조 확인")
print("=" * 60)

print(f"\n🥉 Bronze Layer (원본 데이터):")
try:
    bronze_files = wr.s3.list_objects(BRONZE_LAYER)
    print(f"   파일 수: {len(bronze_files)}개")
except:
    print("   (비어 있음)")

print(f"\n🥈 Silver Layer (정제된 데이터):")
try:
    silver_files = wr.s3.list_objects(SILVER_LAYER)
    print(f"   파일 수: {len(silver_files)}개")
except:
    print("   (비어 있음)")

print(f"\n🥇 Gold Layer (집계된 데이터):")
try:
    gold_files = wr.s3.list_objects(GOLD_LAYER)
    print(f"   파일 수: {len(gold_files)}개")
except:
    print("   (비어 있음)")

print("\n" + "=" * 60)
print("🎉 Lab 1-3: Batch 데이터 파이프라인 완료!")
print("=" * 60)

---

## 🎯 학습 성과

이 실습을 완료하면:

1. ✅ **Data Lake 아키텍처** 이해 (Bronze/Silver/Gold Layer)
2. ✅ **ETL Pipeline** 완전한 이해
3. ✅ **데이터 품질 관리** 자동화 구현
4. ✅ **Batch 데이터 처리** 실무 경험
5. ✅ **AWS Wrangler** 활용 능력
6. ✅ **프로덕션 데이터 파이프라인** 설계 역량

---

## 📚 다음 단계

**Day 2: 모델 서빙 & 버전 관리**
- Lab 2-1: FastAPI 모델 서빙
- Lab 2-2: MLflow Tracking & Registry
- Lab 2-3: KServe 모델 배포

---

© 2025 현대오토에버 MLOps Training